In [1]:
import json
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pandas_ta as ta
import re
import requests

from datetime import datetime

pd.options.display.max_columns = 100
pd.options.display.max_rows = 100

In [2]:
from invaas.schwab.schwab_task import SchwabTask

task = SchwabTask(env="local")
await task.setup_api_and_get_products()

2024-01-29 16:40:48 WARNING No DBUtils defined in the runtime
2024-01-29 16:40:48 INFO Initializing task for local environment
2024-01-29 16:40:48 INFO Historical period used for analysis: 14 days
2024-01-29 16:40:48 INFO Min buy amount: $5
2024-01-29 16:40:48 INFO Max buy amount: $100
2024-01-29 16:40:48 INFO Logging in to Schwab


In [3]:
def get_options_chains_data(ticker: str):
    options_info = requests.get(f"https://client.schwab.com/symlup/OptionsCwp/Underlying/{ticker}.txt").text
    options_info = re.sub(r"[\n\t\r]*", "", options_info)
    options_info = json.loads(options_info.replace('SuggestionBox.JsonpCallback("option",', "")[:-1])
    expirations = [x["Date"] for x in options_info["Expirations"]]
    options_chain_url = f"https://ausgateway.schwab.com/api/is.CSOptionChainsWeb/v1/OptionChainsPort/OptionChains/chains?Symbol={ticker}&IncludeGreeks=false&{','.join(expirations)}"
    task.schwab_api.update_token("update")
    return json.loads(requests.get(options_chain_url, headers=task.schwab_api.headers).text)


options_chains_data = get_options_chains_data("QQQ")
options_chains_data

{'UnderlyingData': {'Ask': '424.5',
  'AskSize': '12',
  'Bid': '424.49',
  'BidSize': '1',
  'Close': '423.81',
  'CumulativeVolume': '11906014',
  'Description': 'INVSC QQQ TRUST SRS 1',
  'ErrorCode': 0,
  'InstrumentType': 'ETF',
  'Last': '424.5',
  'NetChange': '0.69',
  'PercentChange': '0.16',
  'Open': '424.27',
  'DailyHigh': '425.12',
  'DailyLow': '423.61',
  'High52W': '429.85',
  'Low52W': '285.19',
  'Symbol': 'QQQ',
  'DividendYield': 0.5,
  'ExchangeName': 'NASDAQ',
  'IsDelayed': False},
 'Expirations': [{'ExpirationGroup': {'RootSymbol': 'QQQ',
    'MonthAndDay': 'Jan 29',
    'Year': '2024',
    'Day': 'Mon',
    'SettlementType': '',
    'DaysUntil': 0},
   'AdjustedLabel': {},
   'AdjustedGroup': '',
   'Chains': [{'SymbolGroup': 'QQQ20240129353',
     'Legs': [{'Sym': 'QQQ   240129C00353000',
       'OptionType': 'C',
       'Strk': '353',
       'Chg': '-0.3',
       'ChgPct': '-0.42',
       'Bid': '71.42',
       'Ask': '71.51',
       'Lst': '71.37',
       '

In [4]:
data = []
underlying_last = float(options_chains_data["UnderlyingData"]["Last"])

for expiration in options_chains_data["Expirations"]:
    days_until_expiration = expiration["ExpirationGroup"]["DaysUntil"]
    expiration_month_day = expiration["ExpirationGroup"]["MonthAndDay"].split(" ")
    expiration_month = expiration_month_day[0]
    expiration_day = expiration_month_day[1]
    expiration_year = expiration["ExpirationGroup"]["Year"]
    expiration_date = datetime.strptime(f"{expiration_month} {expiration_day} {expiration_year}", "%b %d %Y")

    for chain in expiration["Chains"]:
        data_obj = {}
        data_obj["OPTION_ID"] = chain["SymbolGroup"]
        data_obj["UNDERLYING_LAST"] = underlying_last
        data_obj["EXPIRE_DATE"] = expiration_date
        data_obj["DTE"] = days_until_expiration

        for leg in chain["Legs"]:
            data_obj["STRIKE"] = float(leg["Strk"])
            data_obj["STRIKE_DISTANCE"] = abs(data_obj["STRIKE"] - underlying_last)
            data_obj["STRIKE_DISTANCE_PCT"] = data_obj["STRIKE_DISTANCE"] / underlying_last
            leg_prefix = leg["OptionType"] + "_"
            data_obj[leg_prefix + "ID"] = leg["Sym"]
            data_obj[leg_prefix + "BID"] = float(leg["Bid"])
            data_obj[leg_prefix + "ASK"] = float(leg["Ask"])
            data_obj[leg_prefix + "VOLUME"] = int(leg["Vol"])

        data.append(data_obj)

df_options_chains = pd.DataFrame(data=data)
df_options_chains

,OPTION_ID,UNDERLYING_LAST,EXPIRE_DATE,DTE,STRIKE,STRIKE_DISTANCE,STRIKE_DISTANCE_PCT,C_ID,C_BID,C_ASK,C_VOLUME,P_ID,P_BID,P_ASK,P_VOLUME
0,QQQ20240129353,424.5,2024-01-29,0,353.0,71.5,0.168433,QQQ 240129C00353000,71.42,71.51,1,QQQ 240129P00353000,0.0,0.01,5
1,QQQ20240129354,424.5,2024-01-29,0,354.0,70.5,0.166078,QQQ 240129C00354000,70.42,70.51,0,QQQ 240129P00354000,0.0,0.01,0
2,QQQ20240129355,424.5,2024-01-29,0,355.0,69.5,0.163722,QQQ 240129C00355000,69.42,69.51,13,QQQ 240129P00355000,0.0,0.01,0
3,QQQ20240129356,424.5,2024-01-29,0,356.0,68.5,0.161366,QQQ 240129C00356000,68.42,68.51,0,QQQ 240129P00356000,0.0,0.01,0
4,QQQ20240129357,424.5,2024-01-29,0,357.0,67.5,0.159011,QQQ 240129C00357000,67.42,67.51,0,QQQ 240129P00357000,0.0,0.01,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4183,QQQ20261218620,424.5,2026-12-18,1054,620.0,195.5,0.460542,QQQ 261218C00620000,10.36,15.00,0,QQQ 261218P00620000,193.0,197.90,0
4184,QQQ20261218625,424.5,2026-12-18,1054,625.0,200.5,0.472320,QQQ 261218C00625000,9.62,14.50,0,QQQ 261218P00625000,198.0,202.90,0
4185,QQQ20261218630,424.5,2026-12-18,1054,630.0,205.5,0.484099,QQQ 261218C00630000,9.00,13.50,0,QQQ 261218P00630000,203.0,207.90,0
4186,QQQ20261218635,424.5,2026-12-18,1054,635.0,210.5,0.495878,QQQ 261218C00635000,10.65,11.00,30,QQQ 261218P00635000,208.0,212.89,0


In [22]:
from invaas.schwab.cnn_fear_greed_index import get_historical_cnn_fear_greed_index


def get_fear_greed_index_data():
    historical_period = 14
    df = pd.DataFrame(data=get_historical_cnn_fear_greed_index()["data"])
    df.set_index(
        pd.DatetimeIndex([pd.Timestamp(x, unit="ms", tz="UTC") for x in df.x]),
        inplace=True,
    )
    df.rename(columns={"y": "fear_greed_index"}, inplace=True)
    df.sort_index(inplace=True)
    df["fear_greed_index"] = df.fear_greed_index.fillna(method="ffill").astype(float)
    df["previous_max_fear_greed_index"] = (
        df["fear_greed_index"].rolling(window=historical_period, min_periods=historical_period).max()
    )
    df["previous_min_fear_greed_index"] = (
        df["fear_greed_index"].rolling(window=historical_period, min_periods=historical_period).min()
    )
    return (
        int(df.fear_greed_index.values[-1]),
        int(df.previous_max_fear_greed_index.values[-1]),
        int(df.previous_min_fear_greed_index.values[-1]),
    )


current_fear_greed_index, previous_max_fear_greed_index, previous_min_fear_greed_index = get_fear_greed_index_data()

print(f"Current fear greed index: {current_fear_greed_index}")
print(f"Previous max fear greed index: {previous_max_fear_greed_index}")
print(f"Previous min fear greed index: {previous_min_fear_greed_index}")
print()

good_call_buy = current_fear_greed_index >= previous_max_fear_greed_index - 2
good_put_buy = current_fear_greed_index <= previous_min_fear_greed_index + 2

print(f"Buy call: {good_call_buy}")
print(f"Buy put: {good_put_buy}")

Current fear greed index: 75
Previous max fear greed index: 77
Previous min fear greed index: 61

Buy call: True
Buy put: False


In [6]:
min_dte = 14
min_volume = 100
max_strike_distance_pct = 0.1
max_buy_price = 100

df_options_to_buy = df_options_chains.loc[
    (df_options_chains.DTE > min_dte)
    & (df_options_chains.STRIKE_DISTANCE_PCT < max_strike_distance_pct)
    & (
        (good_call_buy & (df_options_chains.C_VOLUME > min_volume) & (df_options_chains.C_ASK * 100 < max_buy_price))
        | (good_put_buy & (df_options_chains.P_VOLUME > min_volume) & (df_options_chains.P_ASK * 100 < max_buy_price))
    )
].sort_values(by="STRIKE_DISTANCE_PCT")
df_options_to_buy

,OPTION_ID,UNDERLYING_LAST,EXPIRE_DATE,DTE,STRIKE,STRIKE_DISTANCE,STRIKE_DISTANCE_PCT,C_ID,C_BID,C_ASK,C_VOLUME,P_ID,P_BID,P_ASK,P_VOLUME
1514,QQQ20240216444.78,424.5,2024-02-16,18,444.78,20.28,0.047774,QQQ 240216C00444780,0.86,0.87,201,QQQ 240216P00444780,20.45,20.56,0
1515,QQQ20240216445,424.5,2024-02-16,18,445.00,20.50,0.048292,QQQ 240216C00445000,0.83,0.84,113,QQQ 240216P00445000,20.64,20.77,0
1517,QQQ20240216450,424.5,2024-02-16,18,450.00,25.50,0.060071,QQQ 240216C00450000,0.44,0.45,272,QQQ 240216P00450000,25.48,25.60,2
2026,QQQ20240315460,424.5,2024-03-15,46,460.00,35.50,0.083628,QQQ 240315C00460000,0.94,0.95,124,QQQ 240315P00460000,35.39,35.60,0
1725,QQQ20240301465,424.5,2024-03-01,32,465.00,40.50,0.095406,QQQ 240301C00465000,0.24,0.25,205,QQQ 240301P00465000,40.39,40.58,0


In [18]:
positions = task.schwab_api.get_balance_positions_v2()
owned_option_positions = [
    x for x in positions["positionDetails"]["positions"] if x["securityType"] == "Option" and x["shares"] > 0
]
owned_call_options = [x["symbol"] for x in owned_option_positions if x["displaySymbol"].endswith("C")]
owned_put_options = [x["symbol"] for x in owned_option_positions if x["displaySymbol"].endswith("P")]
print(owned_call_options)
print(owned_put_options)
positions

[]
[]


{'brokerageAccountId': 47004171,
 'accountType': 0,
 'dayTradeCount': 1,
 'isPatternDayTrader': False,
 'pendingPurchasesEnabled': True,
 'showPendingPurchasesBanner': False,
 'openOrdersIncluded': True,
 'balanceDetails': {'availableToTradeBalances': {'cash': 504.79,
   'settledFunds': 504.79,
   'cashWithBorrow': 504.79,
   'sma': 47.0,
   'netWorth': 598.66},
  'marginableSecurityBalances': {'equitiesAndEtfs': 504.79,
   'mutualFunds': 504.79,
   'shortSelling': 504.79},
  'nonMarginableSecurityBalances': {'equitiesAndEtfs': 504.79,
   'mutualFunds': 504.79,
   'pennyStocks': 504.79},
  'optionsBalances': {'longOptions': 504.79, 'shortOptions': 555.43}},
 'positionDetails': {'positions': [{'symbol': 'AAPL',
    'displaySymbol': None,
    'symbolDescription': 'APPLE INC',
    'securityType': 'Stock/ETF',
    'itemIssueId': 1973757747,
    'shares': 0.0973,
    'shortShares': 0.0,
    'totalShares': 0.0973,
    'totalOptions': 0.0,
    'reinvestDividend': True,
    'reinvestCapitalGai

In [19]:
df_owned_options_chains = df_options_chains.loc[
    (df_options_chains.C_ID.isin(owned_call_options)) | (df_options_chains.P_ID.isin(owned_put_options))
]
df_owned_options_chains

,OPTION_ID,UNDERLYING_LAST,EXPIRE_DATE,DTE,STRIKE,STRIKE_DISTANCE,STRIKE_DISTANCE_PCT,C_ID,C_BID,C_ASK,C_VOLUME,P_ID,P_BID,P_ASK,P_VOLUME


In [20]:
task.schwab_api.trade_v2(
    ticker=df_owned_options_chains.C_ID.values[0], asset_class="Option", side="Sell", qty=1, dry_run=False
)

IndexError: index 0 is out of bounds for axis 0 with size 0